In [82]:
import pandas as pd 
import numpy as np 
import pickle
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout,BatchNormalization,Conv1D,GlobalAveragePooling1D,Activation, Embedding

training_set=pd.read_csv('../dataset/labeledTrainData.tsv',sep='\t')
training_set.head(2)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."


In [83]:
# train_test_split
y_train=training_set['sentiment'].values
x_train=training_set['review'].values
train_data,test_data,train_labels,test_labels=train_test_split(x_train,y_train,shuffle=True,test_size=0.2,random_state=42,stratify=y_train)

In [84]:
y_train = [[1,0] if y==0 else [0,1] for y in train_labels ]
y_test = [[1,0] if y==0 else [0,1] for y in test_labels ]

y_train = np.array(y_train)
y_test = np.array(y_test)

In [85]:
max_features = 8192
maxlen = 128
embed_size = 64

In [86]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_data)

# save tokenizer for testing
import pickle
with open('tokenizer.sav','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

token_train = tokenizer.texts_to_sequences(train_data)
token_test = tokenizer.texts_to_sequences(test_data)

x_train = pad_sequences(token_train, maxlen=maxlen, padding='post')
x_test = pad_sequences(token_test, maxlen=maxlen, padding='post')

In [87]:
input = Input(shape=(maxlen,))
net = Embedding(max_features, embed_size)(input)
net = Dropout(0.2)(net)
net = BatchNormalization()(net)
net = Conv1D(32, 7, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
net = Conv1D(32, 3, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
net = Conv1D(32, 3, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
net = Conv1D(32, 3, padding='same', activation='relu')(net)
net1 = BatchNormalization()(net)
net = Conv1D(2, 1)(net)
net = GlobalAveragePooling1D()(net)
output = Activation('softmax')(net)
model = Model(inputs = input, outputs = output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [88]:
# model.summary()

In [89]:
model.fit(x_train, y_train, batch_size=2048, epochs=5, validation_split=0.1)

Epoch 1/5
9/9 [==============================] - 29s 3s/step - loss: 0.6777 - acc: 0.5854 - val_loss: 0.6933 - val_acc: 0.4925
Epoch 2/5
9/9 [==============================] - 30s 3s/step - loss: 0.5198 - acc: 0.7649 - val_loss: 0.6775 - val_acc: 0.6065
Epoch 3/5
9/9 [==============================] - 32s 4s/step - loss: 0.3394 - acc: 0.8561 - val_loss: 0.6354 - val_acc: 0.8350
Epoch 4/5
9/9 [==============================] - 31s 3s/step - loss: 0.2433 - acc: 0.8988 - val_loss: 0.6686 - val_acc: 0.4950
Epoch 5/5
9/9 [==============================] - 31s 3s/step - loss: 0.1874 - acc: 0.9266 - val_loss: 0.6928 - val_acc: 0.4930


In [90]:
model.evaluate(x_test, y_test)

157/157 [==============================] - 2s 14ms/step - loss: 0.6868 - acc: 0.5004


[0.6867843866348267, 0.5004000067710876]

In [91]:
test_input = tokenizer.texts_to_sequences('hello')
print(test_input)

test_input = pad_sequences(test_input, maxlen=maxlen, padding='post')
# print(test_input)

model.predict(test_input)

[[1902], [945], [2006], [2006], [1715]]


array([[0.6360091 , 0.36399093],
       [0.6359386 , 0.36406142],
       [0.6382874 , 0.36171263],
       [0.6382874 , 0.36171263],
       [0.6367879 , 0.3632121 ]], dtype=float32)